In [1]:
import pandas as pd
import numpy as np
import json
import requests
from pandas.io.json import json_normalize 
from geopy.geocoders import Nominatim

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.cm as cm
import matplotlib.colors as colors

import folium
import re

print('Libraries imported.')

Libraries imported.


In [2]:
html_data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
canada_df = pd.DataFrame(html_data[0])
canada_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h3>Cleaning the data

In [3]:
canada_df = canada_df[canada_df.Borough != 'Not assigned']

<h3>Grouping Neighborhoods by ',' instead of a '/'

In [4]:
canada_df['Neighborhood'] = canada_df['Neighborhood'].apply(lambda x : re.sub(' / ',',',x))

<h3>Replacing not assigned neighborhoods with borough names

In [5]:
canada_df['Neighborhood'] = canada_df['Neighborhood'].where((canada_df['Neighborhood'] != 'Not assigned'), canada_df['Borough'])
canada_df.reset_index(drop=True,inplace=True)
canada_df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h3> Using a csv file with location information

In [6]:
coord = pd.read_csv('Geospatial_Coordinates.csv')
coord.set_index(['Postal Code'], inplace=True)
canada_df.set_index(['Postal Code'], inplace=True)
canada_lat_long = canada_df.join(coord,how='left')
canada_lat_long.reset_index(inplace=True)
canada_lat_long.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<h3>Let's look at all the neighborhoods in Canada on the map

In [7]:
locator = Nominatim(user_agent="myAgent", timeout=10)
location = locator.geocode('Toronto, Canada')


canada_map = folium.Map([location.latitude,location.longitude],zoom_start=11)
for neigh, lat,long in zip(canada_lat_long['Neighborhood'],canada_lat_long['Latitude'],canada_lat_long['Longitude']):
    label = folium.Popup(neigh, parse_html=True)
    folium.CircleMarker([lat,long], popup=label, parse_html=True).add_to(canada_map)

canada_map

<h3> Initializing data for the Foursquare API

In [8]:
CLIENT_ID = 'GMINQGQGTEJRL5PV23OJH45IFASRCIYGCBKOJICSTLV431MH' # your Foursquare ID
CLIENT_SECRET = 'LL0FGH4MLG4BLKLCYVGPRDCJBQVSLE2DVO21HPKZCZ4A5LQL' # your Foursquare Secret
VERSION = '20200430'
LIMIT = 10

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GMINQGQGTEJRL5PV23OJH45IFASRCIYGCBKOJICSTLV431MH
CLIENT_SECRET:LL0FGH4MLG4BLKLCYVGPRDCJBQVSLE2DVO21HPKZCZ4A5LQL


In [9]:
def getItalianRestaurants(names, latitudes, longitudes):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                'Italian Restaurant', 
                LIMIT)

        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude']
    
    return(nearby_venues,results)
    

In [10]:
cannada_venues,res = getItalianRestaurants(names = canada_lat_long['Neighborhood'], latitudes=
                                 canada_lat_long['Latitude'], longitudes=canada_lat_long['Longitude'])
cannada_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
0,Parkwoods,43.753259,-79.329656,VIA CIBO | italian streetfood,43.754067,-79.357951
1,Parkwoods,43.753259,-79.329656,Sorento Restaurant,43.726575,-79.341989
2,Parkwoods,43.753259,-79.329656,Paisano,43.754946,-79.349580
3,Parkwoods,43.753259,-79.329656,Remezzo Italian Bistro,43.778649,-79.308264
4,Parkwoods,43.753259,-79.329656,Beiruti Grand Cafe,43.773002,-79.332533


<h3>Let's look at all the italian restaurants on the map
   

In [15]:
for name,lat,long in zip(cannada_venues['Venue'],cannada_venues['Venue Latitude'],cannada_venues['Venue Longitude']):
    label = folium.Popup(name, parse_html=True)
    folium.CircleMarker([lat,long], popup=label,fill=True, fill_color='red',color='red', parse_html=True, radius=2).add_to(canada_map)

canada_map

<p>As you can see Toronto has many italian restaurants and there are certain neighborhoods that have a higher concentration of restaurants

<h3>Let's find the number of restaurants per neighborhood in descending order

In [25]:
count_df=cannada_venues.groupby('Neighborhood').count()
count_df_sorted=count_df.sort_values(by='Venue',ascending=False)
count_df_sorted.head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Neighborhood,,,,,
Downsview,40,40,40,40,40
Don Mills,20,20,20,20,20
Willowdale,20,20,20,20,20
Parkwoods,10,10,10,10,10
Scarborough Village,10,10,10,10,10


<p>We can see that Downsview has the maximum number of Italian Restaurants in Toronto, which means that this is an italian restaurant hotspot. This information can be leveraged to our need. 

<hr>

<p>The most popular place for italian restaurants is generally the best entry point to show off our speciality. <br>Let's look at the top 5 neighborhoods that have the highest concentration of Italian Restaurants. 

In [26]:
count_df_sorted.reset_index(inplace=True)


In [67]:
top_5_neighborhoods = list(count_df_sorted.iloc[:5,0])
print("The top 5 neighborhoods are :")
print(top_5_neighborhoods)

The top 5 neighborhoods are :
['Downsview', 'Don Mills', 'Willowdale', 'Parkwoods', 'Scarborough Village']


<p> Another entry point could be the places with the least concentration of Italian Restaurant.
    <br>Let's look at the bottom 5 neighborhoods

In [68]:
bottom_5_neighborhoods = list(count_df_sorted.tail(5).iloc[:,0])
print("The bottom 5 nrighborhoods are: ")
bottom_5_neighborhoods.reverse()
print(bottom_5_neighborhoods)

The bottom 5 nrighborhoods are: 
['Woburn', 'Cedarbrae', 'York Mills, Silver Hills', 'First Canadian Place, Underground city', 'Forest Hill North & West']


<h3>Visualizing the data on the map

In [70]:
final_map = folium.Map([location.latitude,location.longitude],zoom_start=11)
for neigh, lat,long in zip(canada_lat_long['Neighborhood'],canada_lat_long['Latitude'],canada_lat_long['Longitude']):
    label = folium.Popup(neigh, parse_html=True)
    if(neigh in top_5_neighborhoods ):
        folium.CircleMarker([lat,long], popup=label,color='green',fill=True,fill_color='green', parse_html=True).add_to(final_map)
    elif(neigh in bottom_5_neighborhoods):
        folium.CircleMarker([lat,long], popup=label,color='yellow',fill=True,fill_color='yellow', parse_html=True).add_to(final_map)        
    else:
        folium.CircleMarker([lat,long], popup=label,radius=2, parse_html=True).add_to(final_map)

final_map